# report

In [32]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os
import copy
from contract_select import contract_select
from hedge_calc import hedge_calc
from greek_calc import greek_calc, pnl_daily_calc
from position_adj_cost import position_adj_cost, initial_position_cost

In [33]:
moneyness = 'out'
main_type = 'P'
hedge_type = 'P'
main_minDistance = 0.08
hedge_minDistance = 0.06
main_original = contract_select(select_type='main', date= '2018-12-10', spy_spx='spx', contract_type = main_type, minAsk=1.0, minDistance = main_minDistance, minDTE=15, moneyness=moneyness, num=3)
hedge_original = contract_select(select_type='hedge', date= '2018-12-10', spy_spx='spx', contract_type = hedge_type, minAsk=0.2, minDistance = hedge_minDistance, minDTE=15, moneyness=moneyness, num=3)
main_original.update(hedge_original)

position = hedge_calc(main_original)
initial_spread_cost = initial_position_cost(position)

In [ ]:
greek_pnl_daily = pd.DataFrame()
greek_pnl_daily['daily_pnl'] = 0
greek_pnl_daily['adjust_cost'] = 0

d = copy.deepcopy(position)
daily = []

for i in range(250):
    greek_pnl_daily = greek_pnl_daily.append(greek_calc(d))
    d = pnl_daily_calc(d)
    greek_pnl_daily['daily_pnl'][-1] =  d['daily_rtn']
    daily.append(d)
    # print(d['main'].index[0][0], d['main']['[DTE]'][0], d['daily_rtn'])
    temp = copy.deepcopy(hedge_calc(d))
    
    # drop/wind up certain main contracts that meet some criteria
    date = temp['main'].index[0][0]
    if main_type == 'P': 
        temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] > temp['main']['[STRIKE]']]
        temp['main'] = temp['main'][temp['main']['[P_ASK]'] > 0.5]
    if main_type == 'C': 
        temp['main'] = temp['main'][temp['main']['[UNDERLYING_LAST]'] < temp['main']['[STRIKE]']]
        temp['main'] = temp['main'][temp['main']['[C_ASK]'] > 0.5]
    temp['main'] = temp['main'][temp['main']['[DTE]'] > 5]
    temp['main'] = temp['main'][temp['main']['[STRIKE_DISTANCE_PCT]'] >= main_minDistance/2]
    if len(temp['main']) < 3:
        new_contract = contract_select(select_type='main', date = date, spy_spx='spx', contract_type = main_type, minAsk=0.2, minDistance = main_minDistance, minDTE=15, moneyness=moneyness, num=3-len(temp['main']))
        temp['main'] = temp['main'].append(new_contract['main'])
        temp['main']['WEIGHT'] = -1/len(temp['main'])
    # drop/wind up certain hedge contracts that meet some criteria
    if hedge_type == 'P': temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] > temp['hedge']['[STRIKE]']]
    if hedge_type == 'C': temp['hedge'] = temp['hedge'][temp['hedge']['[UNDERLYING_LAST]'] < temp['hedge']['[STRIKE]']]
    temp['hedge'] = temp['hedge'][temp['hedge']['[DTE]'] > 5]
    temp['hedge'] = temp['hedge'][temp['hedge']['[STRIKE_DISTANCE_PCT]'] >= hedge_minDistance/2]
    if len(temp['hedge']) < 3:
        new_contract = contract_select(select_type='hedge', date = date, spy_spx='spx', contract_type = hedge_type, minAsk=0.2, minDistance = hedge_minDistance, minDTE=15, moneyness=moneyness, num=3-len(temp['hedge']))
        temp['hedge'] = temp['hedge'].append(new_contract['hedge'])
        temp['hedge']['WEIGHT'] = 1/len(temp['hedge'])
        
    greek_pnl_daily['adjust_cost'][-1] = position_adj_cost(d, temp)['adjust_cost']
    d=copy.deepcopy(temp)

In [39]:
greek_pnl_daily.to_clipboard()

In [37]:
daily[-1]['hedge']

[UNDERLYING_LAST]  [DTE]  [STRIKE]  [P_BID]  \
[QUOTE_DATE] [EXPIRE_DATE]                                                
2019-04-03   2019-04-22               2873.47   19.0    2625.0     1.20   
             2019-04-26               2873.47   23.0    2635.0     2.36   
             2019-04-26               2873.47   23.0    2625.0     2.16   

                            [P_ASK]  [P_DELTA]  [P_GAMMA]  [P_VEGA]  \
[QUOTE_DATE] [EXPIRE_DATE]                                            
2019-04-03   2019-04-22        1.35   -0.02481    0.00041   0.39369   
             2019-04-26        2.51   -0.04036    0.00059   0.64787   
             2019-04-26        2.29   -0.03736    0.00059   0.60142   

                            [P_THETA]  [STRIKE_DISTANCE]  \
[QUOTE_DATE] [EXPIRE_DATE]                                 
2019-04-03   2019-04-22      -0.18803              248.5   
             2019-04-26      -0.25762              238.5   
             2019-04-26      -0.24320              248.5   

                            [STRIKE_DISTANCE_PCT]    WEIGHT   PNL  \
[QUOTE_DATE] [EXPIRE_DATE]                                          
2019-04-03   2019-04-22                     0.086 -0.533148  0.00   
             2019-04-26                     0.083  0.439428  0.05   
             2019-04-26                     0.086  0.066086  0.00   

                            PNL_PERCENTAGE  
[QUOTE_DATE] [EXPIRE_DATE]                  
2019-04-03   2019-04-22           0.000000  
             2019-04-26           0.001053  
             2019-04-26           0.000000

In [4]:
position

{'main_type': 'spx',
 'main':                             [UNDERLYING_LAST]  [DTE]  [STRIKE]  [P_BID]  \
 [QUOTE_DATE] [EXPIRE_DATE]                                                
 2018-12-10   2018-12-26               2637.83   16.0    2360.0     2.85   
              2018-12-26               2637.83   16.0    2410.0     4.40   
              2018-12-26               2637.83   16.0    2425.0     5.00   
 
                             [P_ASK]  [P_DELTA]  [P_GAMMA]  [P_VEGA]  \
 [QUOTE_DATE] [EXPIRE_DATE]                                            
 2018-12-10   2018-12-26        3.00   -0.04135    0.00053   0.50269   
              2018-12-26        4.59   -0.06392    0.00078   0.71128   
              2018-12-26        5.20   -0.07238    0.00092   0.78576   
 
                             [P_THETA]  [STRIKE_DISTANCE]  \
 [QUOTE_DATE] [EXPIRE_DATE]                                 
 2018-12-10   2018-12-26      -0.43218              277.8   
              2018-12-26      -0.57168      